In [331]:
import numpy as np
import pandas as pd

import random

from sklearn.datasets import make_classification

In [332]:
#X, y = make_classification(n_samples=500, n_features=20, n_informative=2, random_state=42)
#X = pd.DataFrame(X)
#y = pd.Series(y)
#X.columns = [f'col_{col}' for col in X.columns]

In [333]:
df = pd.read_csv(r'D:\Pythonizm\DS_Datasets\banknote+authentication.zip', header=None)
df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'target']
X, y = df.iloc[:,:4], df['target']

In [334]:
display(X.head())
print(X.shape)

,variance,skewness,curtosis,entropy
0,3.62160,8.6661,-2.8073,-0.44699
1,4.54590,8.1674,-2.4586,-1.46210
2,3.86600,-2.6383,1.9242,0.10645
3,3.45660,9.5228,-4.0112,-3.59440
4,0.32924,-4.4552,4.5718,-0.98880


(1372, 4)


In [335]:
display(y.head())
print(y.shape)

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

(1372,)


In [336]:
class MyTreeClf():

    def __init__(self, max_depth = 5, min_samples_split = 2, max_leafs = 20, bins: int = None, criterion: str = 'entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.bins = bins
        self.criterion = criterion
        self.leafs_cnt = 0
        self.depth = 0


    def __repr__(self):
        return f'MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}, bins={self.bins}'


    def _entropy(self, y):
        return -np.sum([(i/len(y))*np.log2(i/len(y)) if i>0 else 0 for i in y.value_counts(sort=False)])
    

    def _gini(self, y):
        return 1 - (np.sum([(i/len(y))**2 if i>0 else 0 for i in y.value_counts(sort=False)]))


    def _get_best_split(self, X, y):
        X, y = X.copy(), y.copy()
        if self.criterion == 'entropy':
            S0 = self._entropy(y)
        elif self.criterion == 'gini':
            Gp = self._gini(y)
        max_gain = -1e15

        for col in X:
            if self.dividers and type(self.dividers[col]) != bool:
                dividers = self.dividers[col]
            else:
                values = sorted(X[col].unique())
                dividers = [(values[i]+values[i+1])/2 for i in range(len(values)-1)]

            for divider in dividers:
                split_right = y[X[col] > divider]
                split_left = y[X[col] <= divider]
                if self.criterion == 'entropy':
                    S_right = (len(split_right)/len(y))*self._entropy(split_right)
                    S_left = (len(split_left)/len(y))*self._entropy(split_left)
                    gain = S0 - S_right - S_left
                elif self.criterion == 'gini':
                    gini_right = (len(split_right)/len(y))*self._gini(split_right)
                    gini_left = (len(split_left)/len(y))*self._gini(split_left)
                    gain = Gp - gini_left - gini_right
                if gain > max_gain:
                    col_name, split_value, max_gain = col, divider, gain
        return col_name, split_value, max_gain


    def _is_leaf(self, X, y):
        if len(X) == 1:
            return True
        if len(X) < self.min_samples_split:
            return True
        if self._entropy(y) == 0:
            return True
        if self.bins:
            for col in X:
                for var in self.dividers[col]:
                    if sum(X[col]>var)>0 and sum(X[col]<var)>0:
                        return False
            return True
        return False
    

    def _split(self, X: pd.DataFrame, y: pd.Series, data: pd.Series, side: str, depth: int, verbose: bool, lif_t: int):
        if not self._is_leaf(X[data], y[data]) and self.leafs_cnt < (self.max_leafs - 1 - lif_t) and depth < self.max_depth:
            if verbose:
                print(side.lower(), len(X[data]), depth+1)
            self._splitter(X[data], y[data], verbose, depth, lif_t+1)
        else:
            if verbose:
                print(f'\n{side} Done | size: {len(X[data])} | entrope: {self._entropy(y[data])} | depth: {depth+1} | val: {np.sum(y[data])/len(y[data])}\n')
                self.leaf_sum += np.sum(y[data])/len(y[data])
            self.leafs_cnt += 1
            self.model[depth+1][2**(depth) - self.rest[depth+1]] = sum(y[data])/len(y[data])
            for i in range(self.max_depth - depth + 1):
                self.rest[depth + i + 1] -= 2**i


    def _splitter(self, X, y, verbose, depth = 0, lif_t=-1):
        depth += 1
        split = self._get_best_split(X, y)
        self.model[depth][2**(depth-1) - self.rest[depth]] = list(split[:2])
        self.rest[depth] -= 1
        if verbose:
            print(split)
        left = X[split[0]] <= split[1]
        right = X[split[0]] > split[1]

        self._split(X, y, left, 'Left', depth, verbose, lif_t)

        self._split(X, y, right, 'Right', depth, verbose, 0)


    def fit(self, X: pd.DataFrame, y: pd.Series, verbose = False):
        X, y = X.copy(), y.copy()
        self.leafs_cnt = 0
        self.depth = 0
        self.model = {}
        self.rest = {}
        self.leaf_sum = 0
        self.dividers = {}

        if self.bins:
            self.dividers = {}
            for col in X:
                values = sorted(X[col].unique())
                dividers = [(values[i]+values[i+1])/2 for i in range(len(values)-1)]
                hists = np.histogram(X[col], self.bins)[1][1:-1]
                if len(dividers) <= self.bins-1:
                    self.dividers[col] = False
                else:
                    self.dividers[col] = hists

        depth = 0
        while depth < self.max_depth + 1:
            depth += 1
            self.model[depth] = [[0, 0] for _ in range(2**(depth-1))]
            self.rest[depth] = 2**(depth-1)
       
        self._splitter(X, y, verbose)


    def predict_proba(self, X: pd.DataFrame):
        X = X.copy()
        y = []
        for row in X.iterrows():
            index = 0
            for depth in range(1, self.max_depth+2):
                if type(self.model[depth][index]) == type(y):
                    col, val = self.model[depth][index][0], self.model[depth][index][1]
                    if row[1][col] <= val:
                        index *= 2
                    else:
                        index *= 2
                        index += 1
                else:
                    y.append(self.model[depth][index])
                    break
        return pd.Series(y)


    def predict(self, X: pd.DataFrame):
        X = X.copy()
        y_proba = self.predict_proba(X)
        y = y_proba.apply(lambda x: 1 if x > 0.5 else 0)
        return y


    def print_tree(self):
        for layer in self.model:
            print(f'{layer} - {self.model[layer]}')
        print(self.leafs_cnt)
        print(round(self.leaf_sum, 6))

In [337]:
tree = MyTreeClf(max_depth=15, min_samples_split=20, max_leafs=30, bins=6, criterion='gini')
print(tree)

MyTreeClf class: max_depth=15, min_samples_split=20, max_leafs=30, bins=6


In [338]:
tree.fit(X, y, True)

('variance', -0.10864999999999903, 0.2307416940473953)
left 594 2
('skewness', 8.497483333333335, 0.12364243943191297)
left 518 3
('skewness', 4.043366666666667, 0.010831313253352813)
left 467 4
('curtosis', 6.32065, 0.006960438033282557)
left 316 5
('curtosis', 2.4517333333333333, 0.00024034609838174958)

Left Done | size: 237 | entrope: -0.0 | depth: 6 | val: 1.0

right 79 6
('entropy', -1.2163999999999993, 0.024359179782990265)

Left Done | size: 1 | entrope: -0.0 | depth: 7 | val: 0.0

right 78 7
('skewness', -0.41075000000000017, 0.003944773175542357)

Left Done | size: 72 | entrope: -0.0 | depth: 8 | val: 1.0


Right Done | size: 6 | entrope: 0.6500224216483541 | depth: 8 | val: 0.8333333333333334

right 151 5
('skewness', -4.864866666666666, 0.18235544668289877)
left 129 6
('entropy', -1.2163999999999993, 0.00026741181419383427)
left 40 7
('variance', -2.4197999999999995, 0.008750000000000091)

Left Done | size: 35 | entrope: -0.0 | depth: 8 | val: 1.0


Right Done | size: 5 | e

In [339]:
tree.print_tree()

1 - [['variance', -0.10864999999999903]]
2 - [['skewness', 8.497483333333335], ['variance', 2.2025000000000006]]
3 - [['skewness', 4.043366666666667], ['variance', -4.73095], ['curtosis', -1.4171833333333335], ['entropy', -1.2163999999999993]]
4 - [['curtosis', 6.32065], ['variance', -2.4197999999999995], 1.0, 0.0, ['skewness', 8.497483333333335], ['entropy', 0.6165500000000002], ['skewness', 8.497483333333335], 0.0]
5 - [['curtosis', 2.4517333333333333], ['skewness', -4.864866666666666], 1.0, 0.0, [0, 0], [0, 0], [0, 0], [0, 0], ['skewness', 4.043366666666667], 0.0, ['skewness', 4.043366666666667], ['curtosis', 2.4517333333333333], ['curtosis', -1.4171833333333335], 0.0, [0, 0], [0, 0]]
6 - [1.0, ['entropy', -1.2163999999999993], ['entropy', -1.2163999999999993], ['skewness', -0.41075000000000017], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], 1.0, ['entropy', -4.882299999999999], [0, 0], [0, 0], ['curtosis', 2.4517333333333333], 0.0, 

In [340]:
tree.predict_proba(X)

0       0.000000
1       0.133333
2       0.000000
3       0.000000
4       0.000000
          ...   
1367    1.000000
1368    1.000000
1369    1.000000
1370    1.000000
1371    1.000000
Length: 1372, dtype: float64

In [341]:
tree.predict(X)

0       0
1       0
2       0
3       0
4       0
       ..
1367    1
1368    1
1369    1
1370    1
1371    1
Length: 1372, dtype: int64